In [1]:
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

2024-05-29 18:17:06.644738: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
device = torch.device("cuda")

In [3]:
processor = AutoProcessor.from_pretrained("HuggingFaceM4/idefics2-8b")
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceM4/idefics2-8b",
).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
image1 = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")
image2 = load_image("https://nofilmschool.com/media-library/t-rex-attack-jurassic-park-scene-breakdown-no-film-school-video-essay.jpg?id=34077360&width=1245&height=700&quality=90&coordinates=0%2C0%2C0%2C1")

In [5]:
# Create inputs
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "What do we see in this image?"},
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "In this image, we can see the city of New York, and more specifically the Statue of Liberty."},
        ]
    },
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "And how about this image?"},
        ]
    },
]
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

In [6]:
inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}


# Generate
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)

In [7]:
from pprint import pprint

pprint(generated_texts)

['User: What do we see in this image? \n'
 'Assistant: In this image, we can see the city of New York, and more '
 'specifically the Statue of Liberty. \n'
 'User: And how about this image? \n'
 'Assistant: In this image we can see a dinosaur, vehicles, poles, and trees.']
